<a href="https://colab.research.google.com/github/mr7495/Pistachio-Counting/blob/master/Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Counting Algorithm from https://github.com/mr7495/Pistachio-Counting

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
!pip install git+https://github.com/mr7495/RetinaNet --upgrade

In [0]:
!git clone https://github.com/mr7495/RetinaNet

In [0]:
import keras
from keras_retinanet import models
import cv2
import numpy as np
import time
import tensorflow as tf
import csv
import shutil
from keras_retinanet.preprocessing.generator import Generator
from keras import backend
import os
from keras_retinanet.utils.eval import _get_detections
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
import random

In [0]:
cd RetinaNet

In [0]:
!python setup.py build_ext --inplace

In [0]:
cd ..

In [0]:
def distance(a,b):
  mida=[(a[0]+a[2])/2,(a[1]+a[3])/2]
  midb=[(b[0]+b[2])/2,(b[1]+b[3])/2]
  dist=np.sqrt((mida[0]-midb[0])**2+(mida[1]-midb[1])**2)
  return(dist)

In [0]:
#This function extracts the frames of the videos
def frame_generator(video_path):
  cap = cv2.VideoCapture(video_path)
  count = 0
  data=[]
  try:
    shutil.rmtree('frames')
  except:
    pass
  try:
    os.mkdir('frames')
  except:
    pass
  while cap.isOpened():
      ret,frame = cap.read()
      if ret is True:
          image=cv2.resize(frame,(1070,600))
          cv2.imwrite("frames/%d.jpg" % count, image)
          data.append(["%d.jpg" % count,0,0,20,20,0])
          count = count + 1
      else:
          break
  cap.release()
  with open('frames/names.csv', mode='w',newline='') as csvfile:
      csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
      for d in data:
          csv_writer.writerow(d)  
  with open('frames/classes.csv', mode='w',newline='') as csvfile:
      csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
      csv_writer.writerow([0,0])  
      csv_writer.writerow([1,1]) 
  

In [0]:
#This function will get the detections from the trained netwrok
def get_detections(generator,model):
  numbers=0
  for r,d,f in os.walk('frames'):
      for file in f:
        if '.jpg' in file:
          numbers+=1
  all_boxes=[]
  all_labels=[]
  for i in range(generator.size()):
    if i%50==0:
      print(i,'Frames Processed/',numbers,'Frames')
    raw_image    = generator.load_image(i)
    image        = generator.preprocess_image(raw_image.copy())
    image, scale = generator.resize_image(image)

    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

    # correct for image scale
    boxes /= scale
    indices= np.argwhere(scores[0]>=0.5)
    boxes=boxes[0][indices]
    scores=scores[0][indices]
    labels=labels[0][indices]
    boxes=boxes.copy()
    boxes=boxes.reshape((np.shape(boxes)[0],np.shape(boxes)[2]))
    all_boxes.append(boxes)
    all_labels.append(labels)
  return(all_boxes,all_labels)

In [0]:
#Counting Algorithm
def count(model,video_path,all_boxes,all_labels):
  # first we have to set the inital threshold
  start=time.time()
  previous=0 #number of pistachios in the previous frame
  current=0 #number of pistachios in the current frame
  boxesb=[] # pistachios detected boxes in previous frame
  max_dist=20 # the maximum allowed distance for assignment
  first_inputs=[] #List of the new inserted pistachios
  for i in range(len(all_boxes)):
    boxesl=all_boxes[i] # pistachios detected boxes in the current frame
    labels=all_labels[i] # labels of the detected pistachios in the current frame
    current=len(boxesl) #number of pistachios in the current frame
    boxess=[] # list of the pistachios in the previous and current frames that should be investigated for assignment
    selected_inds=[] #list of the assigned pistachios
    #Here we assign the pistachios in previous and current frames
    if current > previous:
      for nbl,bl in enumerate(boxesl):
        distances=[]
        boxess=[]
        for nbb,bb in enumerate(boxesb):
          distances.append(distance(bl,bb))
          boxess.append([nbl,nbb])
        if len(distances)==0:
          continue
        if min(distances)<=max_dist:
          bbl,bbb =boxess[np.argmin(distances)]
          selected_inds.append(bbl)
    else:
      for nbl,bl in enumerate(boxesl):
        distances=[]
        boxess=[]
        for nbb,bb in enumerate(boxesb):
          distances.append(distance(bl,bb))
          boxess.append([nbl,nbb])
        if len(distances)==0:
          continue
        if min(distances)<=max_dist:
          bbl,bbb =boxess[np.argmin(distances)]
          selected_inds.append(bbl)

    
    for index in range(len(boxesl)):
        if index not in selected_inds: #if a detected pistachio in the current frame not be assigned to any pistachios in the previous frame
          new_box= boxesl[index] #Get the detected box of this pistachio
          mid_p=[(new_box[0]+new_box[2])/2,(new_box[1]+new_box[3])/2] #compute the centeral point of the detected box
          if mid_p[1] < 200: #if the height of the centeral point be less than 200 (our image heigh is 600)
            first_inputs.append(mid_p[1]) #this pistachio will be considered as a new input
    boxesb=boxesl.copy()
    previous=current
  initial_threshold=np.average(first_inputs) #the average of the new inputs heights would be set as initial threshold

  # After setting the inital and end threshhold, counting begins
  previous=0
  current=0
  boxesb=[]
  tracks_list=[] #The list of the tracked pistachios
  indexesb={} #This dictionary indicates that each of the pistachios in the previous frame belongs to which track list
  end_threshold=500
  max_dist=20
  count=0 #number of all the pistachios
  for i in range(len(all_boxes)):
    boxesl=all_boxes[i]
    labels=all_labels[i]
    current=len(boxesl)
    boxess=[]
    selected_inds=[]  #selected pistachios for assignment in the current frame
    selected_inds_b=[] #selected pistachios for assignment in the previous frame
    indexes={} #This dictionary indicates that each of the pistachios in the current frame belongs to which track list
    losts=[] #The Lost-Pistachios List
    # Here assignment will be performed
    if current > previous:
      for nbl,bl in enumerate(boxesl):
        distances=[]
        boxess=[]
        for nbb,bb in enumerate(boxesb):
          distances.append(distance(bl,bb))
          boxess.append([nbl,nbb])
        if len(distances)==0:
          continue
        if min(distances)<=max_dist:
          bbl,bbb =boxess[np.argmin(distances)]
          selected_inds.append(bbl)
          selected_inds_b.append(bbb)
    else:
      for nbb,bb in enumerate(boxesb):
        distances=[]
        boxess=[]
        for nbl,bl in enumerate(boxesl):
          distances.append(distance(bl,bb))
          boxess.append([nbl,nbb])
        if len(distances)==0:
          continue
        if min(distances)<=max_dist:
          bbl,bbb =boxess[np.argmin(distances)]
          selected_inds.append(bbl)
          selected_inds_b.append(bbb)
      
    for ns,s in enumerate(selected_inds_b):
      if indexesb[s]=='r': # check if the pistachio in the previous frame not be in exiting area  
        indexes[selected_inds[ns]]='r' #indicate that this pistachio should not be added to the track list
      else:
        bbox=boxesl[selected_inds[ns]] #compute the current frame pistachio box
        bbox_y=(bbox[1]+bbox[3])/2 #compute the box mid-height
        if bbox_y<end_threshold: # check if the pistachio in the current frame not be in exiting area    
          tracks_list[indexesb[s]].append([i,bbox,bbox_y,labels[selected_inds[ns]]])
          indexes[selected_inds[ns]]=indexesb[s]
        else: # reject to be added to the track list  
          tracks_list[indexesb[s]].append(['END']) #finish the track list
          indexes[selected_inds[ns]]='r'
    #Here we detect the new inserted pistachios   
    for index in range(len(boxesl)): 
      if index not in selected_inds: #if the current frame pistachio is not assigned
        new_box= boxesl[index] #get the box of the pistachio
        mid_p=[(new_box[0]+new_box[2])/2,(new_box[1]+new_box[3])/2] #compute the box mid-point
        if mid_p[1] < initial_threshold: #if the height of the mid-point be less than the initial threshold
          if current > previous: #if the number of pistachios have been increased
            count+=1 #count one pistachio as new inpur
            tracks_list.append([[i,new_box,mid_p[1],labels[index]]]) #the new-inserted pistachio begins a new track in the tracks list
            indexes[index]=len(tracks_list)-1 #write the track number that this new-inserted pistachio has started
          else:
            losts.append([i,new_box,mid_p[1],labels[index],index]) #the pistachio will not be considered as a new input and will be added to the losts list
        elif mid_p[1]>=end_threshold: #if the pistachio be in the exiting area
          indexes[index]='r' #reject to be added to the track list
        elif mid_p[1]<end_threshold and mid_p[1]>initial_threshold: #be between the entering and exiting area
          losts.append([i,new_box,mid_p[1],labels[index],index]) #add to losts list

    for lost in losts: #try to assign the pistachios in the losts list to the pistachios in the 2 to 6 previous frames
      possible1=[]
      possible2=[]
      possible3=[]
      possible4=[]
      possible5=[]
      started_frame=lost[0] #the frame the lost pistachio is in it
      for nd,detail in enumerate(tracks_list):
        if detail[-1][0]=='END': #not check the ended pistachios (those that were in the exiting area)
          continue
        ended_frame=detail[-1][0] 
        if ended_frame==started_frame-2:
          possible1.append(nd) #2 previous frames
        elif ended_frame==started_frame-3:
          possible2.append(nd) #3 previous frames
        elif ended_frame==started_frame-4:
          possible3.append(nd) #4 previous frames
        elif ended_frame==started_frame-5:
          possible4.append(nd) #5 previous frames
        elif ended_frame==started_frame-6:
          possible5.append(nd) #6 previous frames
      distances=[]
      for p1 in possible1:
        distances.append(distance(tracks_list[p1][-1][1],lost[1]))
      if len(distances)==0: #if the distances list is empty continue
        distances.append(1000)
      if min(distances)<=30: 
        joinable_details_index =possible1[np.argmin(distances)] #add the pistachios that can be joined
        tracks_list[joinable_details_index].append(lost[:-1])
        indexes[lost[-1]]=joinable_details_index
      else: #not joined
        distances=[]
        for p2 in possible2:
          distances.append(distance(tracks_list[p2][-1][1],lost[1]))
        if len(distances)==0:
          distances.append(1000)
        if min(distances)<=45:
          joinable_details_index =possible2[np.argmin(distances)]
          tracks_list[joinable_details_index].append(lost[:-1])
          indexes[lost[-1]]=joinable_details_index
        else:        
          distances=[]
          for p3 in possible3:
            distances.append(distance(tracks_list[p3][-1][1],lost[1]))
          if len(distances)==0:
            distances.append(1000)
          if min(distances)<=60:
            joinable_details_index =possible3[np.argmin(distances)]
            tracks_list[joinable_details_index].append(lost[:-1])
            indexes[lost[-1]]=joinable_details_index
          else:
            distances=[]
            for p4 in possible4:
              distances.append(distance(tracks_list[p4][-1][1],lost[1]))
            if len(distances)==0:
              distances.append(1000)
            if min(distances)<=75:
              joinable_details_index =possible4[np.argmin(distances)]
              tracks_list[joinable_details_index].append(lost[:-1])
              indexes[lost[-1]]=joinable_details_index
            else:
              distances=[]
              for p5 in possible5:
                distances.append(distance(tracks_list[p5][-1][1],lost[1]))
              if len(distances)==0:
                distances.append(1000)
              if min(distances)<=90:
                joinable_details_index =possible5[np.argmin(distances)]
                tracks_list[joinable_details_index].append(lost[:-1])
                indexes[lost[-1]]=joinable_details_index
              else: 
                indexes[lost[-1]]='r'

  

    indexesb=indexes.copy()
    boxesb=boxesl.copy()
    previous=current
  stop=time.time()
  print(stop-start)
  o_count=0 #open-mouth pistachios
  for track_list in tracks_list:
    for track in track_list: 
      if track[-1][0]==1: #if there is at least one open-mouth pistachio in a track list
        o_count+=1 #consider the track list as an open-mouth pistachio
        break

  print('Counted All: ',count) ##number of all the pistachios
  print("Counted Open: ",o_count) #number of open-mouth pistachios
  print("Counted Closed: ",count-o_count) #number of closed-mouth pistachios
  return [count,o_count,count-o_count]


In [0]:
#Load the trained model
model_name='drive/My Drive/Pistachio-trained-nets/resnet152_fold1.h5' #from the shared link in the repo, add the trained network to your drive and run the code
if 'resnet50' in  model_name:
  bn='resnet50'
elif 'resnet152' in  model_name:
  bn='resnet152'
if 'vgg16' in  model_name:
  bn='vgg16'
tmodel = models.load_model(model_name,backbone_name=bn)
model = models.convert_model(tmodel)

In [0]:
#extract the frames and detect the pistachios in them
frame_generator('drive/My Drive/Pesteh-Set/p50-20.mov') #from the shared link in the repo, add the videos folder to your drive and run the code
generator = CSVGenerator('frames/names.csv','frames/classes.csv') 
all_boxes,all_labels=get_detections(generator,model)

In [0]:
#Count
count(model,'drive/My Drive/Pesteh-Set/p50-20.mov',all_boxes,all_labels)